## Mutual Information, Self-information and Split Learning

In this exercise we want to see how split-learning is helping privacy by decreasing the information content in a raw input. This is a simplified example, just to help the understanding of the concepts in this lesson. 

We will be using MNIST data for this exercise. We will first download and load the data. Then, we will load a pretrained small DNN. Our aim is to compare the information in the raw inputs of the MNIST test set, with the information in the output of the final convolution layer, to see if there is information degredation.

In [ ]:
import torch
from torch import nn

from lenet_5 import LeNet5_5
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np

#### Load data

In [ ]:
BATCH_SIZE = 256
BATCH_TEST_SIZE = 1024
data_train = MNIST('.data/mnist',
                   download=True,
                   transform=transforms.Compose([
                       transforms.Resize((32, 32)),
                       transforms.ToTensor()]))
data_test = MNIST('.data/mnist',
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))
data_train_loader = DataLoader(data_train, batch_size = BATCH_SIZE , shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test,  batch_size = BATCH_TEST_SIZE, num_workers=8)
data_test_loader2 = DataLoader(data_test,  batch_size = 1, num_workers=0)

TRAIN_SIZE = len(data_train_loader.dataset)
TEST_SIZE = len(data_test_loader.dataset)
NUM_BATCHES = len(data_train_loader)
NUM_TEST_BATCHES = len(data_test_loader)

#### Load pre-trained model

In [ ]:
model_loaded = LeNet5_5()
model_loaded.load_state_dict(torch.load("./LeNet-saved-5"))
criterion = nn.NLLLoss()

#### Validate

In [ ]:
def validate (net, criterion):
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_test_loader):
        labels = (labels > 5).long()
        output = net(images)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(data_test)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_test)))
    return 

#### Run validate to check the accuracy of the pretrained model

In [ ]:
validate (model_loaded, criterion)

In [ ]:
model_loaded

### Splitting and measuring information content

At this point, we want to split the network to two parts, and observe how different the information content of the original images and the intermediate activations are. We chose the last convolution layer of the pre-trained model we had as the splitting point. We will feed all the test data to the convolutions, and save their outputs so that we can later use them to quantitatively measure the bits of information. 

#### Save the raw images and the intermediate activations

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
imgs = []
intermediate_activations = []
total_correct = 0

model_loaded.eval()
with torch.no_grad():
    for i, (images, labels) in tqdm(enumerate(data_test_loader2), total=len(data_test_loader2)):
        imgs.append(images.squeeze(0).view(1,-1))

        x = model_loaded.convnet(images)
        intermediate_activations.append(x.view(1,120))

    np.save("images", torch.cat(imgs).numpy())
    np.save("intermediate_act", torch.cat(intermediate_activations).numpy())

#### Load the Information Toolbox

We'll be using the Information toolbox to calculate mutual information. Let's load it here. If the `ite-repo` folder isn't in the same directory where you're running this notebook, change the file path to the correct location below.

In [ ]:
import sys
sys.path.insert(1,'./ite-repo')
import ite

Then we'll load the raw images and intermediate activations as Numpy arrays.

In [ ]:
images_raw=np.load("images.npy")
print(images_raw.shape)
intermediate_activation=np.load("intermediate_act.npy")
print(intermediate_activation.shape)

#### Mutual Information function

We will calculate self-information and mutual information of the first 1,000 test images. You can remove the slicing to calculate on full 10,000 test images but it will take longer to run.

$$
I(X;Y) = \sum\limits_{(x,y) \in \mathcal{X}\times\mathcal{Y}} P_{XY}(x,y) \log \frac{P_{XY}(x,y)}{P_X(x)P_Y(y)}
$$

In [ ]:
co = ite.cost.MIShannon_DKL()

Here we'll calculate the self-information of the raw images.

In [ ]:
ds = np.array([1024, 1024])
y = np.concatenate((images_raw[:1000], images_raw[:1000]),axis=1)
print(y.shape)
i = co.estimation(y, ds) 
print(i)

Then we'll calculate the mutual information between the raw images and the intermediate activations.

In [ ]:
ds = np.array([1024, 120])
y = np.concatenate((images_raw[:1000], intermediate_activation[:1000]),axis=1)
print(y.shape)
i = co.estimation(y, ds) 
print(i)

We can see that the raw image contained 455 bits of self-information, whereas the intermediate activations only contain 163 bits of information that was originally in the raw image (the 455 bits). This shows that the first layers of the neural network, alone, have degraded more than half of the original information in the raw input. 